In [5]:
!nvidia-smi

Sun Sep 19 02:20:56 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P8    35W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
! rm -rf object-detection-api && git clone https://github.com/CatchZeng/object-detection-api.git

Cloning into 'object-detection-api'...
remote: Enumerating objects: 129, done.
remote: Counting objects: 100% (129/129), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 129 (delta 37), reused 119 (delta 27), pack-reused 0
Receiving objects: 100% (129/129), 1.65 MiB | 24.17 MiB/s, done.
Resolving deltas: 100% (37/37), done.


In [7]:
! cd object-detection-api && make install

if [ -d "./models" ]; then \
    echo 'models downloaded'; \
  else \
        git clone --depth=1 https://github.com/tensorflow/models; \
fi
Cloning into 'models'...
remote: Enumerating objects: 2920, done.
remote: Counting objects: 100% (2920/2920), done.
remote: Compressing objects: 100% (2448/2448), done.
remote: Total 2920 (delta 740), reused 1274 (delta 434), pack-reused 0
Receiving objects: 100% (2920/2920), 32.97 MiB | 23.08 MiB/s, done.
Resolving deltas: 100% (740/740), done.
cd models/research && \
protoc object_detection/protos/*.proto --python_out=. && \
cp object_detection/packages/tf2/setup.py . && \
python -m pip install .
Processing /content/object-detection-api/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this 

In [8]:
! cd object-detection-api/workspace/test-mask/ && make dl-model

mkdir -p pre-trained-models; \
model=mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8; \
curl -O http://download.tensorflow.org/models/object_detection/tf2/20200711/$model.tar.gz; \
tar zxvf $model.tar.gz; \
mv -f $model ./pre-trained-models/; \
rm -rf $model $model.tar.gz;
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  462M  100  462M    0     0   141M      0  0:00:03  0:00:03 --:--:--  141M
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/checkpoint/
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/checkpoint/ckpt-0.data-00000-of-00001
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/checkpoint/checkpoint
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/checkpoint/ckpt-0.index
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/pipeline.config
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/

In [ ]:
! cd object-detection-api/workspace/test-mask && make train

python model_main_tf2.py \
--model_dir=models/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8 \
--pipeline_config_path=models/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/pipeline.config
2021-09-19 02:22:01.903323: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-19 02:22:01.913787: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-19 02:22:01.914604: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-19 02:22:01.916288: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative va

In [ ]:
! cd object-detection-api/workspace/test-mask && make export && python test_images.py